In [11]:
import kagglehub
from kagglehub import KaggleDatasetAdapter

file_name = "diabetes_prediction_dataset.csv"

df = kagglehub.dataset_load(
    KaggleDatasetAdapter.PANDAS,
    "iammustafatz/diabetes-prediction-dataset",
    file_name,
)

If you want to see details about exploratory data analysis, please take a look in the file [exploratory_data_analysis.ipynb](exploratory_data_analysis.ipynb) instead, because this file will be focused in get a baselines model.

In [9]:
# Convert the column "blood_glucose_level" to float
df['blood_glucose_level'] = df['blood_glucose_level'].astype(float)

In [16]:
# Transform feature gender in binary (Male == 1, Female = 0)
def is_male(gender: str):
    return 1 if gender.strip().lower() == 'male' else 0
df['gender'] = df['gender'].map(is_male)

In [20]:
# Create a feature to get the age group with advanced age
df['is_advanced_age'] = (df['age'] > 65).astype(int)
# Remove the column "age" from the dataset
df = df.drop(columns=['age'])

In [23]:
# Transform the info smoking_history in information if the patient is a smoker or not
df['is_smoker'] = df['smoking_history'].map({'never': 0, 'No Info': 0, 'current': 1, 'former': 1, 'ever': 1, 'not current': 0})
# Remove the column "age" from the dataset
df = df.drop(columns=['smoking_history'])

In [24]:
df

,gender,hypertension,heart_disease,bmi,HbA1c_level,blood_glucose_level,diabetes,is_advanced_age,is_smoker
0,0,0,1,25.19,6.6,140,0,1,0
1,0,0,0,27.32,6.6,80,0,0,0
2,1,0,0,27.32,5.7,158,0,0,0
3,0,0,0,23.45,5.0,155,0,0,1
4,1,1,1,20.14,4.8,155,0,1,1
...,...,...,...,...,...,...,...,...,...
99995,0,0,0,27.32,6.2,90,0,1,0
99996,0,0,0,17.37,6.5,100,0,0,0
99997,1,0,0,27.83,5.7,155,0,1,1
99998,0,0,0,35.42,4.0,100,0,0,0


In [ ]:
# Select features of type number/float from the dataset
converted_df = df.select_dtypes(['number', 'float'])